## Load latest headlines

In [1]:
# ========== Bootstrap: ensure required Python packages are present =========
import importlib, subprocess, sys
from typing import Optional

def _ensure(pkg_name: str, import_name: Optional[str] = None):
    try:
        importlib.import_module(import_name or pkg_name)
    except ModuleNotFoundError:
        subprocess.check_call([sys.executable, '-m', 'pip', 'install', pkg_name])
    finally:
        globals()[import_name or pkg_name] = importlib.import_module(import_name or pkg_name)

_ensure('pandas')
print('All dependencies ready.\n')


All dependencies ready.



In [2]:
from pathlib import Path
archive = Path('archive')
archive.mkdir(exist_ok=True)
import pandas as pd
latest = pd.read_csv('../headlines/latest.csv')
latest.head()

,pubdate,source,title,link
0,2025-11-19-10-50-41 +0000,nyt,Trump Administration Gives Three Mile Island N...,https://www.nytimes.com/2025/11/19/us/nuclear-...
1,2025-11-19-10-43-00 +0000,wsj,Netherlands Hands Back Control of Chip Maker N...,https://www.wsj.com/world/china/dutch-governme...
2,2025-11-19-10-39-58 +0000,nyt,"These Books Were Judged by Their A.I. Covers, ...",https://www.nytimes.com/2025/11/19/books/new-z...
3,2025-11-19-10-35-43 +0000,nypost,Illegal immigrant accused of grabbing ICE offi...,https://nypost.com/2025/11/19/us-news/illegal-...
4,2025-11-19-10-34-35 +0000,bbc,Russian attack hits flats in one of deadliest ...,https://www.bbc.com/news/articles/cy0yere4k0zo...


## Calculate word frequencies

In [3]:
import re
from collections import Counter
from datetime import datetime

with open('exclude.txt') as f:
    stop_words = set(w.strip() for w in f if w.strip())
words = re.findall(r'[A-Za-z]+', ' '.join(latest['title']).lower())
filtered = [w for w in words if w not in stop_words and len(w) > 1]
counts = Counter(filtered)
score_df = (
    pd.DataFrame(counts.items(), columns=['word','score'])
    .sort_values('score', ascending=False)
)
score_df[['score','word']].to_csv('scores.csv', index=False)
timestamp = datetime.utcnow().strftime('%Y-%m-%d-%H-00-00')
score_df[['score','word']].to_csv(archive / f'scores-{timestamp}.csv', index=False)
score_df.head()


/tmp/ipykernel_2334/2220774760.py:15: DeprecationWarning: datetime.datetime.utcnow() is deprecated and scheduled for removal in a future version. Use timezone-aware objects to represent datetimes in UTC: datetime.datetime.now(datetime.UTC).
  timestamp = datetime.utcnow().strftime('%Y-%m-%d-%H-00-00')


,word,score
0,trump,59
296,epstein,25
165,new,19
204,saudi,19
297,files,15


## Rank headlines by score

In [4]:
word_scores = dict(score_df[['word','score']].values)
latest['score'] = latest['title'].apply(
    lambda t: sum(
        word_scores.get(w.lower(), 0)
        for w in re.findall(r'[A-Za-z]+', t)
        if len(w) > 1
    )
)
ranked = latest.sort_values('score', ascending=False)
ranked[['score','pubdate','source','title','link']].to_csv('rank.csv', index=False)
ranked[['score','pubdate','source','title','link']].to_csv(archive / f'rank-{timestamp}.csv', index=False)
ranked.head()


,pubdate,source,title,link,score
347,2025-11-18-13-24-46 +0000,cbc,Saudi crown prince MBS unaware of Khashoggi ki...,https://www.cbc.ca/news/world/washington-saudi...,153
172,2025-11-18-22-58-00 +0000,wsj,Congress’s approval of legislation to force a ...,https://www.wsj.com/politics/policy/epstein-fi...,150
191,2025-11-18-22-15-00 +0000,wsj,"Once one of Trump’s most strident MAGA allies,...",https://www.wsj.com/politics/policy/how-marjor...,139
124,2025-11-19-01-33-54 +0000,nypost,Trump announces Saudi Arabia will be designate...,https://nypost.com/2025/11/18/us-news/trump-an...,138
136,2025-11-19-00-58-13 +0000,cbc,"Once shunned, Saudi crown prince is Trump’s g...",https://www.cbc.ca/news/world/saudi-arabia-cro...,132


## Select top headlines

In [5]:
with open('exclude_sources.txt') as f:
    excluded_sources = {
        line.strip()
        for line in f
        if line.strip() and not line.strip().startswith('#')
    }

top_count = 10
top_rows = []
working = word_scores.copy()
remaining = latest.copy()

while len(top_rows) < top_count and not remaining.empty:
    scored_remaining = remaining.assign(
        score=remaining['title'].apply(
            lambda t: sum(
                working.get(w.lower(), 0)
                for w in re.findall(r'[A-Za-z]+', t)
                if len(w) > 1
            )
        )
    )
    ranked_loop = scored_remaining.sort_values('score', ascending=False)

    if ranked_loop.empty:
        break

    top_story = ranked_loop.iloc[0]
    remaining = remaining.drop(top_story.name)

    if top_story['source'] in excluded_sources:
        continue

    words = set(re.findall(r'[A-Za-z]+', top_story['title'].lower()))
    for word in words:
        working.pop(word, None)

    top_rows.append(top_story[['score', 'pubdate', 'source', 'title', 'link']])

top_df = pd.DataFrame(top_rows)
top_df.to_csv('top.csv', index=False)
top_df.to_csv(archive / f'top-{timestamp}.csv', index=False)
top_df


,score,pubdate,source,title,link
347,153,2025-11-18-13-24-46 +0000,cbc,Saudi crown prince MBS unaware of Khashoggi ki...,https://www.cbc.ca/news/world/washington-saudi...
172,91,2025-11-18-22-58-00 +0000,wsj,Congress’s approval of legislation to force a ...,https://www.wsj.com/politics/policy/epstein-fi...
48,63,2025-11-19-08-42-26 +0000,nypost,Family of high school California tennis star k...,https://nypost.com/2025/11/19/us-news/parents-...
191,55,2025-11-18-22-15-00 +0000,wsj,"Once one of Trump’s most strident MAGA allies,...",https://www.wsj.com/politics/policy/how-marjor...
264,43,2025-11-18-18-04-02 +0000,nyt,Zelensky Will Try to Revive Peace Talks With R...,https://www.nytimes.com/2025/11/18/world/europ...
202,40,2025-11-18-21-46-47 +0000,latimes,"'Played with fire, got burned': GOP control of...",https://www.latimes.com/politics/story/2025-11...
254,39,2025-11-18-18-52-13 +0000,nypost,Google’s Sundar Pichai warns of AI spending ‘i...,https://nypost.com/2025/11/18/business/google-...
49,36,2025-11-19-08-38-08 +0000,wsj,Stock Market Today: S&P 500 Futures Inch Up as...,https://www.wsj.com/livecoverage/nvidia-earnin...
280,35,2025-11-18-17-18-09 +0000,nypost,NY Gov. Kathy Hochul enlists Wall Street execs...,https://nypost.com/2025/11/18/business/ny-gov-...
299,33,2025-11-18-16-20-13 +0000,cbc,North Carolina becomes latest battleground as ...,https://www.cbc.ca/news/world/us-north-carolin...


## Create JSON version of top headlines

In [6]:
import pandas as pd
pd.read_csv('top.csv').to_json('top.json', orient='records', indent=2)
